In [9]:
import pandas as pd
import numpy as np

In [2]:
csv_file = 'output_knee_treatment(AG6-14-2022).csv'
df = pd.read_csv(csv_file)
to_drop_columns = []
for column in df.keys():
    if column.startswith('Unnamed'):
        to_drop_columns.append(column)
df.drop(columns=to_drop_columns, inplace=True)
df.head()

RRN PrimaryDiagnosis        DOB  AgeNow  EncountersAge  \
0  334233504   Cerebral Palsy  18-Apr-10      12             10   
1  334233504   Cerebral Palsy  18-Apr-10      12             10   
2  334233504   Cerebral Palsy  18-Apr-10      12             10   
3  334233504   Cerebral Palsy  18-Apr-10      12             10   
4  334233504   Cerebral Palsy  18-Apr-10      12             10   

  Encounters.Date  TestNumber                 GcdFile   Side  \
0       21-May-20           2  AW0220 braces06-R1.gcd  Right   
1       21-May-20           2  AW0220 braces06-R1.gcd  Right   
2       21-May-20           2  AW0220 braces06-R1.gcd  Right   
3       21-May-20           2  AW0220 braces06-R1.gcd  Right   
4       21-May-20           2                     NaN  Right   

   LeftKneeRomPopAngle  ...  RightThighFootAngle  \
0                   70  ...                  NaN   
1                   70  ...                  NaN   
2                   70  ...                  NaN   
3                   70  ...                  NaN   
4                   70  ...                  NaN   

  AnkleDorsiPlantarMeanStanceDF KneeFlexionAtIC KneeFlexionMin HipFlexionMin  \
0                      5.699855       20.876699       1.888724    -21.060829   
1                      5.699855       20.876699       1.888724    -21.060829   
2                      5.699855       20.876699       1.888724    -21.060829   
3                      5.699855       20.876699       1.888724    -21.060829   
4                      5.073318       21.680480       4.601491    -18.689020   

  Procedures.Date Procedures.Side                 Code  \
0       21-May-20               L  Gastroc Lengthening   
1       21-May-20               L               Hams-M   
2       21-May-20               L         Prox-Fem-DRO   
3       21-May-20               L          PT-Transfer   
4       21-May-20               L  Gastroc Lengthening   

                     Treatment  \
0  No Treatment Recommendation   
1  No Treatment Recommendation   
2  No Treatment Recommendation   
3  No Treatment Recommendation   
4  No Treatment Recommendation   

                                                Path  
0  Cerebral Palsy -> Age Encounter -> 5 <= Age <=...  
1  Cerebral Palsy -> Age Encounter -> 5 <= Age <=...  
2  Cerebral Palsy -> Age Encounter -> 5 <= Age <=...  
3  Cerebral Palsy -> Age Encounter -> 5 <= Age <=...  
4  Cerebral Palsy -> Age Encounter -> 5 <= Age <=...  

[5 rows x 30 columns]

In [3]:
df['Code'].unique()

array(['Gastroc Lengthening', 'Hams-M', 'Prox-Fem-DRO', 'PT-Transfer',
       'HW-Removal', 'BTX-FCU', 'BTX-Gast', 'BTX-Hams', 'Foot-Other',
       'Pat tendon advance', 'Cal Sliding', 'Hip - Other', 'OTHER',
       'wed-osteo-calcan', 'Strayer-TAL', 'Plant-Fascia', 'TAL',
       'Calc Med Sliding Ost', 'LLCast', 'Tib-DRO-IR', 'SLC',
       'Botox - Other', 'BTX-Pronator', 'Pelvic-Ost', 'V-DRO'],
      dtype=object)

In [4]:
df['Treatment'].unique()

array(['No Treatment Recommendation',
       'Correct Ankle Equinus for Back Kneeing',
       'Hamstring Lengthening & Correct Crouch Causes',
       'Posterior Knee Capsulotomy & Correct all Crouch Causes',
       'No Hip or Foot Treatment RecommendationHamstring Lengthening & Correct Crouch Causes',
       'Knee Capsulotomy & Hamstring Lengthening',
       'No Hip or Foot Treatment RecommendationNo Treatment Recommendation'],
      dtype=object)

In [5]:
### Remove the keyword 'No Hip or Foot Treatment Recommendation'

# df['Treatment'].startswith('No Hip or Foot Treatment Recommendation')

## Map the surgical code and treatment recommendations

In [6]:
mapping_file = 'knee-treatment-surgical-code-mapping.csv'
df_mapping = pd.read_csv(mapping_file, usecols=['Knee Treatment Decision Tree Outcomes', 'Code 1', 'Code 2', 'Code 3', 'Code 4', 'Code 5'])
df_mapping = df_mapping[df_mapping['Knee Treatment Decision Tree Outcomes'].notna()]
df_mapping

Knee Treatment Decision Tree Outcomes           Code 1  \
0                   Botox Injection & Knee Splinting         BTX-Hams   
1                        No Treatment Recommendation              NaN   
2                              Hamstring Lengthening      Ham-Release   
3           Knee Capsulotomy & Hamstring Lengthening      Ham-Release   
4             Correct Ankle Equinus for Back Kneeing     Gast Release   
5  Hip Extension Osteotomy, Planovalgus & Torsion...      Dis-Fem-DRO   
6            No Hip or Foot Treatment Recommendation              NaN   
7      Hamstring Lengthening & Correct Crouch Causes  Dis-Fem-Ext-Ost   
8  Posterior Knee Capsulotomy & Correct all Crouc...  Dis-Fem-Ext-Ost   
9  Knee Extension Osteotomy & Correct all Crouch ...  Dis-Fem-Ext-Ost   

                Code 2        Code 3      Code 4      Code 5  
0               LLCast           NaN         NaN         NaN  
1                  NaN           NaN         NaN         NaN  
2               Hams-M       Hams-ML         NaN         NaN  
3               Hams-M       Hams-ML         NaN         NaN  
4  Gastroc Lengthening   Strayer-TAL         TAL         NaN  
5         Prox-Fem-DRO  Prox-Fem-DRO  Tib-DRO-ER  Tib-DRO-IR  
6                  NaN           NaN         NaN         NaN  
7   Pat tendon advance           NaN         NaN         NaN  
8   Pat tendon advance           NaN         NaN         NaN  
9   Pat tendon advance           NaN         NaN         NaN

In [41]:
valid_codes = pd.unique(df_mapping[['Code 1', 'Code 2', 'Code 3', 'Code 4', 'Code 5']].values.ravel('K'))
valid_codes

array(['BTX-Hams', nan, 'Ham-Release', 'Gast Release', 'Dis-Fem-DRO',
       'Dis-Fem-Ext-Ost', 'LLCast', 'Hams-M', 'Gastroc Lengthening',
       'Prox-Fem-DRO', 'Pat tendon advance', 'Hams-ML', 'Strayer-TAL',
       'TAL', 'Tib-DRO-ER', 'Tib-DRO-IR'], dtype=object)

In [71]:
correct_predictions = 0
incorrect_predictions = 0
list_incorrect_treatment, list_incorrect_codes = [], []

for index in df.index:
    thisTreatment = df['Treatment'].iloc[index]
    if thisTreatment.startswith('No Hip or Foot Treatment Recommendation'):
        thisTreatment = thisTreatment.replace('No Hip or Foot Treatment Recommendation', '')
    mapping_index = np.where(df_mapping['Knee Treatment Decision Tree Outcomes'] == thisTreatment)
    
    ## if not find in mapping df
    if not np.any(mapping_index):
        print(index)
        print(thisTreatment)
        
    ### If treatment recommendation is 'No Treatment Recommendation'
    if thisTreatment == 'No Treatment Recommendation':
        if df['Code'].iloc[index] not in valid_codes:
            correct_predictions += 1
        else:
            incorrect_predictions += 1
        continue
    
    if (df['Code'].iloc[index] == df_mapping['Code 1'].iloc[mapping_index]).bool() or \
        (df['Code'].iloc[index] == df_mapping['Code 2'].iloc[mapping_index]).bool() or \
        (df['Code'].iloc[index] == df_mapping['Code 3'].iloc[mapping_index]).bool() or \
        (df['Code'].iloc[index] == df_mapping['Code 4'].iloc[mapping_index]).bool() or \
        (df['Code'].iloc[index] == df_mapping['Code 5'].iloc[mapping_index]).bool():
        correct_predictions += 1
    else:
        incorrect_predictions += 1
        list_incorrect_treatment.append(thisTreatment)
        list_incorrect_codes.append(df['Code'].iloc[index])
#         print('Recommended Treatment: ', thisTreatment)
#         print('Surgical Code: ', df['Code'].iloc[index])
#         print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')


#     print(thisTreatment)
#     print(df['Code'].iloc[index])
    
#     break

print('Correct Predictions: ', correct_predictions)
print('Incorrect Predictions: ', incorrect_predictions)
print('Accuracy: ', round(correct_predictions/(correct_predictions+incorrect_predictions), 3))

df_incorrect = pd.DataFrame(list(zip(list_incorrect_treatment, list_incorrect_codes)),
               columns =['Treatment Recommendation', 'Surgical Code'])

# df_incorrect

output_csv = 'incorrect_treatment_recommendation_knee_treatment.csv'
df_incorrect.to_csv(output_csv, index=False)

Correct Predictions:  221
Incorrect Predictions:  163
Accuracy:  0.576
